install requirement

In [2]:
! pip install requests beautifulsoup4 python-pptx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 22.0 MB/s eta 0:00:00


In [134]:
slideshow_url = "https://www.slideshare.net/slideshow/thorie-de-graphe/46698343"
end= 20  # put last number of pptx

extract all links of images

In [135]:
import requests
from bs4 import BeautifulSoup

def get_image_link_from_selector(url, selector):
    try:
        # Fetch the HTML content of the SlideShare page
        response = requests.get(url)
        if response.status_code == 200:
            # Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract image link using CSS selector
            image_element = soup.select_one(selector)

            if image_element and 'src' in image_element.attrs:
                image_link = image_element['src']
                return image_link
            else:
                print("Image link not found.")
                return None
        else:
            print(f"Failed to fetch SlideShare page. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage:
css_selector = '#slide-image-0'
image_link = get_image_link_from_selector(slideshow_url, css_selector)
print("Image link:", image_link)


Image link: https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/85/Theorie-de-graphe-1-320.jpg


In [136]:
import requests
from bs4 import BeautifulSoup

def get_element_content(url, selector):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the element using the specified selector
        element = soup.select_one(selector)

        # Return the text content of the element
        if element:
            return element.text.strip()
        else:
            print(f"Element with selector '{selector}' not found on the page.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve content from {url}. Error: {str(e)}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage
selector = '#metadata-toolbar > div > span'

content = get_element_content(url, selector)
print(content.split(" of "))


['1', '22']


In [141]:
def generate_links(base_url, end, replace_str):
    links = []
    for i in range(1,end):
        new_link = base_url.replace(replace_str, f"{i}-2048.jpg")
        new_link = new_link.replace('/85/', "/75/")
        links.append(new_link)
    return links

# Example usage:
base_url = image_link
replace_str = "1-320.jpg"
links = generate_links(base_url,   end, replace_str)

# Print out the generated links
print(links)


['https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-1-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-2-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-3-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-4-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-5-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-6-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-7-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-graphe-8-2048.jpg', 'https://image.slidesharecdn.com/thoriedegraphe-150406165724-conversion-gate01/75/Theorie-de-gr

In [142]:
import os
import requests
import uuid
folder_name = "images"+str(uuid.uuid4())[:8]  # Using the first 8 characters of the UUID for brevity
def download_images(image_urls):
    # Create a unique folder name based on timestamp
    os.makedirs(folder_name, exist_ok=True)

    # Download each image and save to the folder
    for index, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                image_path = os.path.join(folder_name, f"image_{index+1}.jpg")
                with open(image_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {image_path}")
            else:
                print(f"Failed to download {url}. Status code: {response.status_code}")
        except Exception as e:
            print(f"Failed to download {url}. Error: {str(e)}")

# Example list of image URLs


# Download images
download_images(links)
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image

def resize_image(image_path, max_width, max_height):
    try:
        img = Image.open(image_path)
        width, height = img.size

        # Calculate aspect ratio
        aspect_ratio = width / height

        # Determine new dimensions while maintaining aspect ratio
        if width > max_width or height > max_height:
            if aspect_ratio > 1:
                new_width = max_width
                new_height = int(max_width / aspect_ratio)
            else:
                new_height = max_height
                new_width = int(max_height * aspect_ratio)

            # Resize image
            img = img.resize((new_width, new_height), Image.ANTIALIAS)

        return img

    except Exception as e:
        print(f"Failed to resize {image_path}. Error: {str(e)}")
        return None

def create_pdf_from_images(folder_path, output_pdf):
    # Create a PDF canvas
    c = canvas.Canvas(output_pdf, pagesize=letter)

    # Get all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Iterate over image files and add each as a page in the PDF
    for index, image_file in enumerate(image_files):
        try:
            image_path = os.path.join(folder_path, image_file)
            img = resize_image(image_path, 600, 800)  # Adjust max_width and max_height as needed

            if img:
                # Calculate position to center image on page
                width, height = img.size
                x_pos = (letter[0] - width) / 2
                y_pos = (letter[1] - height) / 2

                # Add the image to the PDF canvas
                c.drawInlineImage(img, x_pos, y_pos, width, height)

                # Add a new page for each image (except the first one)
                if index < len(image_files) - 1:
                    c.showPage()

                print(f"Added {image_file} to PDF")

        except Exception as e:
            print(f"Failed to process {image_file}. Error: {str(e)}")

    # Save the PDF file
    c.save()
    print(f"PDF '{output_pdf}' created successfully with images from folder '{folder_path}'.")

# Example folder containing images
folder_path = "../content/"+folder_name
output_pdf = "output_images.pdf"

# Create PDF from images in the folder
create_pdf_from_images(folder_path, output_pdf)



Downloaded images18fa69aa/image_1.jpg
Downloaded images18fa69aa/image_2.jpg
Downloaded images18fa69aa/image_3.jpg
Downloaded images18fa69aa/image_4.jpg
Downloaded images18fa69aa/image_5.jpg
Downloaded images18fa69aa/image_6.jpg
Downloaded images18fa69aa/image_7.jpg
Downloaded images18fa69aa/image_8.jpg
Downloaded images18fa69aa/image_9.jpg
Downloaded images18fa69aa/image_10.jpg
Downloaded images18fa69aa/image_11.jpg
Downloaded images18fa69aa/image_12.jpg
Downloaded images18fa69aa/image_13.jpg
Downloaded images18fa69aa/image_14.jpg
Downloaded images18fa69aa/image_15.jpg
Downloaded images18fa69aa/image_16.jpg
Downloaded images18fa69aa/image_17.jpg
Downloaded images18fa69aa/image_18.jpg
Downloaded images18fa69aa/image_19.jpg
Added image_12.jpg to PDF


<ipython-input-142-5053dffaf690>:51: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((new_width, new_height), Image.ANTIALIAS)


Added image_10.jpg to PDF
Added image_9.jpg to PDF
Added image_18.jpg to PDF
Added image_7.jpg to PDF
Added image_14.jpg to PDF
Added image_5.jpg to PDF
Added image_6.jpg to PDF
Added image_11.jpg to PDF
Added image_3.jpg to PDF
Added image_19.jpg to PDF
Added image_15.jpg to PDF
Added image_13.jpg to PDF
Added image_17.jpg to PDF
Added image_2.jpg to PDF
Added image_1.jpg to PDF
Added image_16.jpg to PDF
Added image_4.jpg to PDF
Added image_8.jpg to PDF
PDF 'output_images.pdf' created successfully with images from folder '../content/images18fa69aa'.


<ipython-input-83-95011851fd8d>:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((new_width, new_height), Image.ANTIALIAS)


Added image_3.jpg to PDF
Added image_2.jpg to PDF
Added image_1.jpg to PDF
PDF 'output_images.pdf' created successfully with images from folder '../content/6fb0e1a1'.


In [75]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.7 MB/s eta 0:00:00


get / predict urls